In [3]:
import polars as pl

In [6]:
df = pl.read_csv("listings.csv")
print(df)

shape: (26_067, 79)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ id        ┆ listing_u ┆ scrape_id ┆ last_scra ┆ … ┆ calculate ┆ calculate ┆ calculate ┆ reviews_ │
│ ---       ┆ rl        ┆ ---       ┆ ped       ┆   ┆ d_host_li ┆ d_host_li ┆ d_host_li ┆ per_mont │
│ i64       ┆ ---       ┆ i64       ┆ ---       ┆   ┆ stings_co ┆ stings_co ┆ stings_co ┆ h        │
│           ┆ str       ┆           ┆ str       ┆   ┆ unt…      ┆ unt…      ┆ unt…      ┆ ---      │
│           ┆           ┆           ┆           ┆   ┆ ---       ┆ ---       ┆ ---       ┆ f64      │
│           ┆           ┆           ┆           ┆   ┆ i64       ┆ i64       ┆ i64       ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 35797     ┆ https://w ┆ 202503191 ┆ 2025-03-2 ┆ … ┆ 1         ┆ 0         ┆ 0         ┆ null     │
│           ┆ ww.airbnb ┆ 50644     ┆ 1         ┆   ┆           ┆      

## Limpieza de columna 'Price'

In [7]:
if not df.is_empty():
    df = df.with_columns(
        pl.col("price")
        .str.replace_all(r"\$","")
        .str.replace_all(r",","")
        .cast(pl.Float64, strict=False)
        .alias("price")
    ).drop_nulls(
        subset = ["price"]
    ).filter(
        pl.col("price") > 0
    )
    print("Columna 'price limpiada y convertida a tipo nunérico usando Polars")


Columna 'price limpiada y convertida a tipo nunérico usando Polars


## Price Average

In [10]:
price_average = df.select(
    pl.col("price").mean()
)
print(price_average)

shape: (1, 1)
┌─────────────┐
│ price       │
│ ---         │
│ f64         │
╞═════════════╡
│ 1665.005982 │
└─────────────┘


## Most common room

In [13]:
ctx = pl.SQLContext(population=df, eager=True)
most_common = ctx.execute(
        """
        SELECT name, COUNT(id) AS count
        FROM population
        GROUP BY name
        ORDER BY count DESC
        LIMIT 1
    """
)
print(most_common)

shape: (1, 2)
┌─────────────────────────────────┬───────┐
│ name                            ┆ count │
│ ---                             ┆ ---   │
│ str                             ┆ u32   │
╞═════════════════════════════════╪═══════╡
│ Blueground | Roma Sur 1 recama… ┆ 54    │
└─────────────────────────────────┴───────┘


## Top 10 neighborhoods

In [15]:
most_popular_hoods = ctx.execute(
        """
        SELECT neighbourhood_cleansed AS neighborhood, COUNT(id) AS count
        FROM population
        GROUP BY neighbourhood_cleansed
        ORDER BY count DESC
        LIMIT 10
    """
)
print(most_popular_hoods)

shape: (10, 2)
┌─────────────────────┬───────┐
│ neighborhood        ┆ count │
│ ---                 ┆ ---   │
│ str                 ┆ u32   │
╞═════════════════════╪═══════╡
│ Cuauhtémoc          ┆ 10434 │
│ Miguel Hidalgo      ┆ 3764  │
│ Benito Juárez       ┆ 2482  │
│ Coyoacán            ┆ 1362  │
│ Álvaro Obregón      ┆ 799   │
│ Venustiano Carranza ┆ 752   │
│ Tlalpan             ┆ 582   │
│ Gustavo A. Madero   ┆ 424   │
│ Iztacalco           ┆ 389   │
│ Azcapotzalco        ┆ 317   │
└─────────────────────┴───────┘


## Top 10 hosts

In [18]:
most_popular_hosts = ctx.execute(
        """
        SELECT host_name AS host, COUNT(host_id) AS count
        FROM population
        GROUP BY host_name
        ORDER BY count DESC
        LIMIT 10
    """
)
print(most_popular_hosts)

shape: (10, 2)
┌────────────┬───────┐
│ host       ┆ count │
│ ---        ┆ ---   │
│ str        ┆ u32   │
╞════════════╪═══════╡
│ Alejandra  ┆ 305   │
│ Alejandro  ┆ 244   │
│ Juan       ┆ 241   │
│ Blueground ┆ 225   │
│ Raul       ┆ 225   │
│ Luis       ┆ 214   │
│ Mr. W      ┆ 201   │
│ Carlos     ┆ 175   │
│ Eduardo    ┆ 173   │
│ Jorge      ┆ 172   │
└────────────┴───────┘


## predicción de precio de un airbnn (si le paso un dato a un modelo, predice el precio, quitando la columna de precio)
## scikit learn o tensorflow